# Creating a local PatentsView clone using DuckDB

## Imports and utilities

In [ ]:
import duckdb
from duckdb import DuckDBPyConnection
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
import yaml

def zipfile_from_url(filename: str, zipped_url: str):
    """Open file from remote ZIP archive."""
    with urlopen(zipped_url) as remote:
        with ZipFile(BytesIO(remote.read())) as file:
            return file.open(filename)

def create_patentsview_table(con: DuckDBPyConnection, pv_database: str, table_name: str):
    """Download and save given PatentsView table in duckdb database, if it doesn't already exist."""
    filename = f"{table_name}.tsv"
    url = f"{pv_database}/{filename}.zip"

    existing_tables = con.sql("show tables").df().name.values

    if table_name not in existing_tables:
        print(f"Loading {table_name} from {url} ...")
        table = con.read_csv(zipfile_from_url(filename, url), delimiter="\t", all_varchar=True, header=True)
        print(f"Saving to duckdb database...")
        table.create(table_name)
        print("Done.")
        existing_tables = con.sql("show tables").df().name.values


## PatentsView database cloning script

Download all PV tables and save them to duckdb database. This is a large download and requires at least 50Gb of free disk space.

In [ ]:
# Create local DuckDB database.
database_name = "patentsview.ddb"
con = duckdb.connect(database_name)

# Loading information about PatentsView data sources from `sources.yml` file.
with open("sources.yml") as file:
    sources = yaml.safe_load(file)

# Replicating all tables in the duckdb database
for type, info in sources.items():
    pv_database = info['database']  # Base URL to use for granted or pre-grant data downloads
    tables = info['tables']  # List of tables
    for table_name in tables:
        create_patentsview_table(con, pv_database, table_name)

## Example usage

Show all tables:

In [ ]:
con.sql("show all tables;").df().head()

Get a single table:

In [ ]:
rawinventor = con.table("g_inventor_not_disambiguated")
rawinventor.limit(5).df()

Joining multiple tables together:

In [ ]:
rawinventor = con.table("g_inventor_not_disambiguated")
rawlocation = con.table("g_location_not_disambiguated")
patent = con.table("g_patent")

rawinventor.join(rawlocation, "rawlocation_id", how="left").join(patent, "patent_id", how="left").limit(5).df()

## Using SQL in a notebook

In [ ]:
# pip install jupysql jupysql duckdb-engine
%load_ext sql
%sql con
%config SqlMagic.displaycon = False

In [ ]:
%%sql
SELECT *
FROM g_patent
LIMIT 5;